<a href="https://colab.research.google.com/github/michaelfarayola7/Data-Science-ML-Projects/blob/main/COMPAS_Reweighing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing necessary libraries

In [1]:
!pip install aif360
!pip install aif360[LawSchoolGPA]
!pip install aif360[Reductions]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv("/content/g_recidivism.csv")

In [4]:
pd.set_option('display.max_columns',100)

## Dataset Preprocessing

In [5]:
data.head()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,0,0,0
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,0,0,0
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,0,0,1
3,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,0,0,0
4,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,0,0,0


In [6]:
data = data.drop(["age","is_recid","decile_score","days_b_screening_arrest"], axis = 1)

In [7]:
data.shape

(6172, 10)

In [8]:
data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,F,Other,Greater than 45,Low,Male,0,0,0,0,0
1,F,African-American,25 - 45,Low,Male,0,1,0,0,0
2,F,African-American,Less than 25,Low,Male,4,1,0,0,1
3,M,Other,25 - 45,Low,Male,0,0,0,0,0
4,F,Caucasian,25 - 45,Medium,Male,14,1,0,0,0


#### Dealing with Categorical Variables

In [9]:
replaceStruct = {
    "age_cat": {"Less than 25": 0, "25 - 45": 1, "Greater than 45": 2},
    "score_text": {"Low": 0, "Medium": 1, "High": 2},
    "sex": {"Male": 1, "Female": 0}
}

data = data.replace(replaceStruct)


In [10]:
data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,F,Other,2,0,1,0,0,0,0,0
1,F,African-American,1,0,1,0,1,0,0,0
2,F,African-American,0,0,1,4,1,0,0,1
3,M,Other,1,0,1,0,0,0,0,0
4,F,Caucasian,1,1,1,14,1,0,0,0


#### Dummy Variable

In [11]:
data1 = pd.get_dummies(data, columns=['race','c_charge_degree'])
#data = pd.get_dummies(data, columns=['sex'])


In [12]:
data1.head()

,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_F,c_charge_degree_M
0,2,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0
1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0
2,0,0,1,4,1,0,0,1,1,0,0,0,0,0,1,0
3,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1
4,1,1,1,14,1,0,0,0,0,0,1,0,0,0,1,0


### Reweighing

In [13]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [14]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

# Define protected attribute names and privileged group
protected_attribute_names = ['sex']
privileged_group = [{'sex': 1}]
unprivileged_group = [{'sex' : 0}]

In [15]:
#Create an AIF360 BinaryLabelDataset
dataset = BinaryLabelDataset(df=data1,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=protected_attribute_names,
                             privileged_protected_attributes=privileged_group)

In [16]:
dataset

               instance weights features                                 \
                                                    protected attribute   
                                 age_cat score_text                 sex   
instance names                                                            
0                           1.0      2.0        0.0                 1.0   
1                           1.0      1.0        0.0                 1.0   
2                           1.0      0.0        0.0                 1.0   
3                           1.0      1.0        0.0                 1.0   
4                           1.0      1.0        1.0                 1.0   
...                         ...      ...        ...                 ...   
6167                        1.0      0.0        1.0                 1.0   
6168                        1.0      0.0        0.0                 1.0   
6169                        1.0      2.0        0.0                 1.0   
6170                     

In [17]:
# Apply Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
dataset_transformed = RW.fit_transform(dataset)

In [18]:
dataset_transformed

               instance weights features                                 \
                                                    protected attribute   
                                 age_cat score_text                 sex   
instance names                                                            
0                      1.046815      2.0        0.0                 1.0   
1                      0.949180      1.0        0.0                 1.0   
2                      0.949180      0.0        0.0                 1.0   
3                      1.046815      1.0        0.0                 1.0   
4                      0.949180      1.0        1.0                 1.0   
...                         ...      ...        ...                 ...   
6167                   1.046815      0.0        1.0                 1.0   
6168                   1.046815      0.0        0.0                 1.0   
6169                   1.046815      2.0        0.0                 1.0   
6170                   0.

In [19]:
# Split the reweighted dataset into train and test sets
dataset_train, dataset_test = dataset_transformed.split([0.7], shuffle=True)

In [20]:
# Define features and target variable
X_train = dataset_train.features
y_train = dataset_train.labels.ravel()
X_test = dataset_test.features
y_test = dataset_test.labels.ravel()

In [21]:
# Train a Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [22]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

In [23]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the reweighted test set:", accuracy)

Accuracy on the reweighted test set: 0.6803455723542117


### Scaling

In [24]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Modeling

In [25]:
# Train a Logistic Regression model
clf = LogisticRegression(solver = 'liblinear')
clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_scaled)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the reweighted test set:", accuracy)

Accuracy on the reweighted test set: 0.6803455723542117


In [26]:
from aif360.metrics import BinaryLabelDatasetMetric

metric = BinaryLabelDatasetMetric(dataset, unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
disparate_impact = metric.disparate_impact()
print("Disparate Impact:", disparate_impact)


Disparate Impact: 0.7330518950023087


In [27]:
from aif360.metrics import BinaryLabelDatasetMetric

metric = BinaryLabelDatasetMetric(dataset_transformed, unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
disparate_impact = metric.disparate_impact()
print("Disparate Impact:", disparate_impact)

Disparate Impact: 0.9999999999999998


### Setting the race as protectd attributes

In [28]:
data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,F,Other,2,0,1,0,0,0,0,0
1,F,African-American,1,0,1,0,1,0,0,0
2,F,African-American,0,0,1,4,1,0,0,1
3,M,Other,1,0,1,0,0,0,0,0
4,F,Caucasian,1,1,1,14,1,0,0,0


In [29]:
filtered_data = data[(data["race"] == "African-American") | (data["race"] == "Caucasian")]

In [30]:
filtered_data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
1,F,African-American,1,0,1,0,1,0,0,0
2,F,African-American,0,0,1,4,1,0,0,1
4,F,Caucasian,1,1,1,14,1,0,0,0
6,M,Caucasian,1,0,0,0,0,0,0,0
7,F,Caucasian,1,0,1,0,0,0,0,0


In [31]:
filtered_data.shape

(5278, 10)

In [32]:
6172-5278

894

In [33]:
replaceStruct = {
    "race": {"African-American": 0, "Caucasian": 1},
    "c_charge_degree": {"F": 1, "M": 0},
}

filtered_data = filtered_data.replace(replaceStruct)

In [34]:
filtered_data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
1,1,0,1,0,1,0,1,0,0,0
2,1,0,0,0,1,4,1,0,0,1
4,1,1,1,1,1,14,1,0,0,0
6,0,1,1,0,0,0,0,0,0,0
7,1,1,1,0,1,0,0,0,0,0


In [35]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

# Define protected attribute names and privileged group
protected_attribute_names = ['race']
privileged_group = [{'race': 1}]
unprivileged_group = [{'race' : 0}]

In [36]:
#Create an AIF360 BinaryLabelDataset
dataset_2 = BinaryLabelDataset(df=filtered_data,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=protected_attribute_names,
                             privileged_protected_attributes=privileged_group)

In [37]:
# Apply Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
dataset_transformed = RW.fit_transform(dataset_2)

In [38]:
# Split the reweighted dataset into train and test sets
dataset_train, dataset_test = dataset_transformed.split([0.7], shuffle=True)

In [39]:
# Define features and target variable
X_train = dataset_train.features
y_train = dataset_train.labels.ravel()
X_test = dataset_test.features
y_test = dataset_test.labels.ravel()

In [40]:
# Train a Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [41]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

In [42]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the reweighted test set:", accuracy)

Accuracy on the reweighted test set: 0.6982323232323232


In [43]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [44]:
# Train a Logistic Regression model
clf = LogisticRegression(solver = 'liblinear')
clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_scaled)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the reweighted test set:", accuracy)

Accuracy on the reweighted test set: 0.6982323232323232


In [45]:
from aif360.metrics import BinaryLabelDatasetMetric

metric = BinaryLabelDatasetMetric(dataset_2, unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
disparate_impact = metric.disparate_impact()
print("Disparate Impact:", disparate_impact)

Disparate Impact: 1.3384228978676935


In [46]:
from aif360.metrics import BinaryLabelDatasetMetric

metric = BinaryLabelDatasetMetric(dataset_transformed, unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
disparate_impact = metric.disparate_impact()
print("Disparate Impact:", disparate_impact)

Disparate Impact: 1.0000000000000002


### Concatenating X_text_scaled and y_pred

In [47]:
X_test_scaled

array([[-1.38320252,  1.21690576, -1.51324202, ..., -0.14650709,
        -0.18567831, -0.24913202],
       [ 0.72295993, -0.82175632, -1.51324202, ..., -0.14650709,
        -0.18567831, -0.24913202],
       [-1.38320252, -0.82175632,  0.00865181, ..., -0.14650709,
        -0.18567831, -0.24913202],
       ...,
       [ 0.72295993, -0.82175632, -1.51324202, ..., -0.14650709,
        -0.18567831,  1.77349149],
       [ 0.72295993,  1.21690576,  0.00865181, ..., -0.14650709,
        -0.18567831, -0.24913202],
       [ 0.72295993,  1.21690576,  1.53054563, ..., -0.14650709,
        -0.18567831, -0.24913202]])

In [48]:
y_pred

array([0., 0., 0., ..., 1., 1., 0.])

In [49]:
# Convert arrays to DataFrames
df1 = pd.DataFrame(y_test, columns=["y_test"])
df2 = pd.DataFrame(y_pred, columns=["Prediction"])

# Concatenate the DataFrames vertically (along rows)
concatenated_df = pd.concat([df1, df2], axis=1)

# Reset the index (optional)
concatenated_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(concatenated_df)

      y_test  Prediction
0        0.0         0.0
1        0.0         0.0
2        0.0         0.0
3        1.0         1.0
4        1.0         1.0
...      ...         ...
1579     0.0         0.0
1580     1.0         0.0
1581     1.0         1.0
1582     0.0         1.0
1583     0.0         0.0

[1584 rows x 2 columns]


In [50]:
# Define protected attribute names and privileged group
protected_attribute_names = ['Prediction']

#Create an AIF360 BinaryLabelDataset
pred_data = BinaryLabelDataset(df=df2,
                             label_names=['Prediction'],
                             protected_attribute_names=protected_attribute_names)

In [51]:
concatenated_df.head()

,y_test,Prediction
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,1.0,1.0
4,1.0,1.0


In [52]:
X_test_scaled

array([[-1.38320252,  1.21690576, -1.51324202, ..., -0.14650709,
        -0.18567831, -0.24913202],
       [ 0.72295993, -0.82175632, -1.51324202, ..., -0.14650709,
        -0.18567831, -0.24913202],
       [-1.38320252, -0.82175632,  0.00865181, ..., -0.14650709,
        -0.18567831, -0.24913202],
       ...,
       [ 0.72295993, -0.82175632, -1.51324202, ..., -0.14650709,
        -0.18567831,  1.77349149],
       [ 0.72295993,  1.21690576,  0.00865181, ..., -0.14650709,
        -0.18567831, -0.24913202],
       [ 0.72295993,  1.21690576,  1.53054563, ..., -0.14650709,
        -0.18567831, -0.24913202]])

####

In [53]:
filtered_data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
1,1,0,1,0,1,0,1,0,0,0
2,1,0,0,0,1,4,1,0,0,1
4,1,1,1,1,1,14,1,0,0,0
6,0,1,1,0,0,0,0,0,0,0
7,1,1,1,0,1,0,0,0,0,0


In [54]:
test_data = pd.DataFrame(X_test, columns=['c_charge_degree','race','age_cat','score_text','sex','priors_count','juv_fel_count','juv_misd_count','juv_other_count'])

In [55]:
test_data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,juv_fel_count,juv_misd_count,juv_other_count
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0
4,0.0,0.0,1.0,2.0,1.0,13.0,0.0,0.0,0.0


In [56]:
# Concatenate the DataFrames vertically (along rows)
concat_test_data_rlabel = pd.concat([test_data, df1], axis=1)

# Reset the index (optional)
concat_test_data_rlabel.reset_index(drop=True, inplace=True)

In [57]:
concat_test_data_rlabel.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,juv_fel_count,juv_misd_count,juv_other_count,y_test
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,2.0,1.0,13.0,0.0,0.0,0.0,1.0


In [58]:
# Concatenate the DataFrames vertically (along rows)
concat_test_data_plabel = pd.concat([test_data, df2], axis=1)

# Reset the index (optional)
concat_test_data_plabel.reset_index(drop=True, inplace=True)

In [59]:
concat_test_data_plabel.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,juv_fel_count,juv_misd_count,juv_other_count,Prediction
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,2.0,1.0,13.0,0.0,0.0,0.0,1.0


In [60]:
concat_test_data_rlabel = concat_test_data_rlabel.astype(int)
concat_test_data_plabel = concat_test_data_plabel.astype(int)

In [61]:
concat_test_data_plabel = concat_test_data_plabel.rename(columns={"Prediction":"y_test"})

In [62]:
# # Define favorable and unfavorable labels
# favorable_label = 1
# unfavorable_label = 0

# # Define protected attribute names and privileged group
# protected_attribute_names = []
# privileged_group = []
# unprivileged_group = []

In [63]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

# Define protected attribute names and privileged group
protected_attribute_names = ['race']
privileged_group = [{'race': 1}]
unprivileged_group = [{'race' : 0}]

In [64]:
#Create an AIF360 BinaryLabelDataset
BLdata1 = BinaryLabelDataset(df=concat_test_data_rlabel,
                             label_names=['y_test'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=protected_attribute_names,
                             privileged_protected_attributes=privileged_group)

In [65]:
#Create an AIF360 BinaryLabelDataset
BLdata2 = BinaryLabelDataset(df=concat_test_data_plabel,
                             label_names=['y_test'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=protected_attribute_names,
                             privileged_protected_attributes=privileged_group)

In [66]:
######

In [67]:
concat_test_data_rlabel

,c_charge_degree,race,age_cat,score_text,sex,priors_count,juv_fel_count,juv_misd_count,juv_other_count,y_test
0,0,1,0,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,3,0,0,0,0
3,1,1,1,1,1,5,0,0,0,1
4,0,0,1,2,1,13,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1579,1,1,2,0,1,0,0,0,0,0
1580,0,0,1,0,1,0,0,0,0,1
1581,1,0,0,0,1,2,0,0,1,1
1582,1,1,1,1,0,5,0,0,0,0


In [68]:
from aif360.metrics import ClassificationMetric

# Make sure dataset_2 and pred_data have consistent features, labels, and scores
metric = ClassificationMetric(BLdata1, BLdata2, privileged_groups=privileged_group, unprivileged_groups=unprivileged_group)
equal_opportunity_difference = metric.equal_opportunity_difference()
print("Equal Opportunity Difference:", equal_opportunity_difference)


Equal Opportunity Difference: 0.21926364234056545


In [69]:
type(dataset_transformed)

aif360.datasets.binary_label_dataset.BinaryLabelDataset

In [70]:
metric = BinaryLabelDatasetMetric(dataset_transformed, privileged_groups=privileged_group, unprivileged_groups=unprivileged_group)
statistical_parity_difference = metric.statistical_parity_difference()
print("Statistical Parity Difference:", statistical_parity_difference)


Statistical Parity Difference: 1.1102230246251565e-16


In [71]:
metric = ClassificationMetric(BLdata1, BLdata2, privileged_groups=privileged_group, unprivileged_groups=unprivileged_group)
average_odds_difference = metric.average_odds_difference()
print("Average Odds Difference:", average_odds_difference)


Average Odds Difference: 0.17022918390757952
